In [12]:
import pandas as pd
import numpy as np
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
# Download necessary files from NLTK:
# punkt -> Tokenization
# stopwords -> Stop words removal
nltk.download("punkt")
nltk.download("stopwords")
# Load the file dataset and preview it
file = pd.read_csv("reviews.csv")
file.head()
# Filter negative file (score of 1 or 2)
neg__file = file[(file["score"] == 1) | (file["score"] == 2)]["content"]

def preprocess_text(text):
    """Performs tokenization, stop word removal, and non-alpha character removal."""
    tokens = word_tokenize(text)
    filtered_tokens = [
        token for token in tokens if token.isalpha() and token.lower() not in stopwords.words("english")
    ]
    return " ".join(filtered_tokens)

# Apply the preprocessing function to the negative file
neg__file_cleaned = neg__file.apply(preprocess_text)

# Store the preprocessed file in a DataFrame
preprocessed_file = pd.DataFrame({"review": neg__file_cleaned})
print(preprocessed_file.head())


# Vectorize the cleaned file using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_file["review"])


# Apply K-means clustering
clust_kmeans = KMeans(n_clusters=5, random_state=500)
pred_labels = clust_kmeans.fit_predict(tfidf_matrix)

# Store the predicted labels in a list
categories = pred_labels.tolist()
preprocessed_file["category"] = categories


# Get the feature names (terms) from the vectorizer
terms = vectorizer.get_feature_names_out()

# List to save the top term for each cluster
topic_terms_list = []

for cluster in range(clust_kmeans.n_clusters):
    # Get indices of file in the current cluster
    cluster_indices = [i for i, label in enumerate(categories) if label == cluster]

    # Sum the TF-IDF scores for each term in the cluster
    cluster_tfidf_sum = tfidf_matrix[cluster_indices].sum(axis=0)
    cluster_term_freq = np.asarray(cluster_tfidf_sum).ravel()

    # Get the top term and its frequency
    top_term_index = cluster_term_freq.argsort()[::-1][0]
    top_term = terms[top_term_index]
    top_term_frequency = cluster_term_freq[top_term_index]

    # Append the results to the list
    topic_terms_list.append({
        "category": cluster,
        "term": top_term,
        "frequency": top_term_frequency
    })

# Store the results in a DataFrame
topic_terms = pd.DataFrame(topic_terms_list)
print(topic_terms)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                              review
0                                   open app anymore
1           begging refund app month nobody replying
2  costly premium version approx Indian Rupees pe...
3  Used keep organized UPDATES made mess things c...
4                                   Dan Birthday Oct


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


   category     term   frequency
0         0      app  189.611081
1         1    tasks   58.099738
2         2     work   48.774678
3         3     good   37.940386
4         4  version   69.153458
